In [ ]:
from bids import BIDSLayout
import numpy as np
import pandas as pd
import shutil as sh
from pathlib import Path
import re
from notebooks.bn_metadata import read_metadata

In [ ]:
layout = BIDSLayout("../..", database_path="../../.pybids", derivatives=True, validate=False)

In [ ]:
WEIGHT="sift2"
def get_participants(participant_file):
    with open(participant_file) as f:
        subs = [match.group(1) for l in f.read().splitlines() if (match := re.match(r'sub-(.*)', l))]
    return subs
def get_files():
    participants = get_participants('../../derivatives/snakedwi-0.1.0/participants.tsv')
    subjects = list(
        get_subj_metadata()["TOPSY ID"]
        .map(subj_to_str)
        [lambda df: df.isin(participants)]
    )
    return layout.get(
        subject=subjects,
        suffix="connectome",
        desc=WEIGHT,
        atlas="bn246"
    )

def get_connectomes():
    return np.dstack([np.loadtxt(file.path, delimiter=",") for file in get_files()])

def get_subjects():
    return [file.get_entities()["subject"] for file in get_files()]

def get_metadata(hem = None):
    metadata = read_metadata()
    if hem in ("L", "R"):
        return metadata[metadata["hemisphere"] == hem].reset_index()
    return metadata.reset_index()

def get_subj_metadata():
    df = pd.read_csv("../../part", sep="\t")
    return df[df["Patient Cat"].isin([1, 2])]

def subj_to_str(subj):
    return f"{subj:03}"
    


### Connectome Data (Left and Right)

In [ ]:
HEMS = ("L", "R", None)
def get_hemispheric_connectomes():
    connectomes = get_connectomes()
    for hem in HEMS:
        index = get_metadata(hem)["Label ID"]
        yield connectomes[(*np.ix_(index, index), ...)]

for connectomes, hem in zip(get_hemispheric_connectomes(), HEMS):
    base = Path(f"results/nbs/{WEIGHT}_{hem or 'pan'}_connectomes")
    if base.exists():
        sh.rmtree(base)
    base.mkdir(exist_ok=True)
    for connectome, sub in zip(np.moveaxis(connectomes, -1, 0), get_subjects()):
        with base.joinpath(sub).with_suffix(".txt").open('w') as f:
            for line in np.matrix(connectome):
                np.savetxt(f, line, fmt="%.10f")



### Node MNI Coordinates (Left and Right)

In [ ]:
for hem in HEMS:
    with Path(f"results/nbs/coords_{hem or 'pan'}.txt").open('w') as f:
        f.write(
            "\n".join(
                " ".join(coord.strip().split(", "))
                for coord in get_metadata(hem)["MNI"]
            )
        )

### Patient Diagnoses

In [ ]:
with Path(f"results/nbs/ddx.txt").open('w') as f:
    subs = list(map(int, get_subjects()))
    df = get_subj_metadata()
    f.write(
        "\n".join(
            df[df["TOPSY ID"].isin(subs)]["Patient Cat"].map({1: "1 0", 2: "0 1"})
        )
    )

### Node Labels

In [ ]:
with Path("results/nbs/node_labels.txt").open("w") as f:
    # Labels the same on both hemispheres
    f.write("\n".join(get_metadata("L")["Name"]))
    